In [1]:
import pandas as pd

In [2]:

import os

In [3]:

cwd = os.getcwd()

In [4]:

#data_path = 'Anand/'

In [5]:

#similar = pd.read_csv('prediction_sheet_approach_2.csv')
similar = pd.read_csv('scaled_prediction_sheet_approach_2.csv')

In [6]:

similar.columns

Index(['ID', 'ProjectID', 'Requirements', 'Class', 'Attractiveness',
       'Efficiency', 'Perspicuity', 'Dependability', 'Stimulation', 'Novelty',
       'Trust', 'Adaptability', 'Usefulness', 'Value', 'Visual Aesthetics',
       'Intuitive Use', 'Trustworthiness of Content', 'Quality of Content',
       'Haptics', 'Acoustics', 'Clarity', 'Response behavior',
       'Response quality', 'Comprehensibility'],
      dtype='object')

In [7]:

labels = ['Attractiveness',
       'Efficiency', 'Perspicuity', 'Dependability', 'Stimulation', 'Novelty',
       'Trust', 'Adaptability', 'Usefulness', 'Value', 'Visual Aesthetics',
       'Intuitive Use', 'Trustworthiness of Content', 'Quality of Content',
       'Haptics', 'Acoustics', 'Clarity', 'Response behavior',
       'Response quality', 'Comprehensibility']

In [8]:

train = similar[['Requirements','Class']]

In [9]:

train['labels'] = [labels for i in train.index]

/srv/conda/envs/saturn/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:

train.head()

,Requirements,Class,labels
0,The system shall refresh the display every 60 ...,PE,"[Attractiveness, Efficiency, Perspicuity, Depe..."
1,The application shall match the color of the s...,LF,"[Attractiveness, Efficiency, Perspicuity, Depe..."
2,If projected the data must be readable. On a...,US,"[Attractiveness, Efficiency, Perspicuity, Depe..."
3,The product shall be available during normal b...,A,"[Attractiveness, Efficiency, Perspicuity, Depe..."
4,If projected the data must be understandable....,US,"[Attractiveness, Efficiency, Perspicuity, Depe..."


In [11]:

train1 = train.assign(names=train.labels.str.split(",")).explode('labels')

In [12]:

train.shape

(969, 3)

In [13]:

train1.columns

Index(['Requirements', 'Class', 'labels', 'names'], dtype='object')

In [14]:

train1 = train1[['Requirements', 'Class','labels']]

In [15]:
train1 = train1.reset_index()

In [16]:

train1.head(10)

,index,Requirements,Class,labels
0,0,The system shall refresh the display every 60 ...,PE,Attractiveness
1,0,The system shall refresh the display every 60 ...,PE,Efficiency
2,0,The system shall refresh the display every 60 ...,PE,Perspicuity
3,0,The system shall refresh the display every 60 ...,PE,Dependability
4,0,The system shall refresh the display every 60 ...,PE,Stimulation
5,0,The system shall refresh the display every 60 ...,PE,Novelty
6,0,The system shall refresh the display every 60 ...,PE,Trust
7,0,The system shall refresh the display every 60 ...,PE,Adaptability
8,0,The system shall refresh the display every 60 ...,PE,Usefulness
9,0,The system shall refresh the display every 60 ...,PE,Value


In [16]:

train1.to_csv('train.csv',index=None)

In [52]:

train1 = pd.read_csv(data_path + 'train.csv')

In [19]:
# !pip install torch
# !pip install transformers

In [18]:

import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [20]:
possible_labels = train1.labels.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [21]:

train1['label'] = train1.labels.replace(label_dict)

In [22]:

from sklearn.model_selection import train_test_split


In [23]:
X_train, X_val, y_train, y_val = train_test_split(train1.index.values, 
                                                train1.labels.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=train1.labels.values)

In [24]:

train1['data_type'] = ['not_set']*train1.shape[0]

train1.loc[X_train, 'data_type'] = 'train'
train1.loc[X_val, 'data_type'] = 'val'

In [25]:

train1.head()

,index,Requirements,Class,labels,label,data_type
0,0,The system shall refresh the display every 60 ...,PE,Attractiveness,0,val
1,0,The system shall refresh the display every 60 ...,PE,Efficiency,1,train
2,0,The system shall refresh the display every 60 ...,PE,Perspicuity,2,val
3,0,The system shall refresh the display every 60 ...,PE,Dependability,3,train
4,0,The system shall refresh the display every 60 ...,PE,Stimulation,4,train


In [26]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    train1[train1.data_type=='train'].Requirements.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    train1[train1.data_type=='val'].Requirements.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train1[train1.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(train1[train1.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/srv/conda/envs/saturn/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [28]:

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [68]:

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 8

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [69]:

from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [70]:

from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [71]:


import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)



In [72]:

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    


In [73]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [75]:

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/2060 [00:00<?, ?it/s]


Epoch 1
Training loss: 2.9986831388427215
Validation loss: 2.9987982295371673
F1 Score (Weighted): 0.0048035672052651746


Epoch 2:   0%|          | 0/2060 [00:00<?, ?it/s]


Epoch 2
Training loss: 2.9987044050855545
Validation loss: 2.9968443421217112
F1 Score (Weighted): 0.004739542686541863


...


Epoch 3
Training loss: 2.997483926837884
Validation loss: 2.99696874160033
F1 Score (Weighted): 0.007214160380679435


Epoch 4:   0%|          | 0/2060 [00:00<?, ?it/s]


Epoch 4
Training loss: 2.9968436623082577
Validation loss: 2.996866920492151
F1 Score (Weighted): 0.0057316969776035


Epoch 5:   0%|          | 0/2060 [00:00<?, ?it/s]


Epoch 5
Training loss: 2.9964878358887237
Validation loss: 2.996866920492151
F1 Score (Weighted): 0.0057316969776035


In [60]:
import torch
torch.cuda.empty_cache()

In [76]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train1 = tokenizer.batch_encode_plus(
    train1.Requirements.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train1['input_ids']
attention_masks_train = encoded_data_train1['attention_mask']
labels_train = torch.tensor(train1.label.values)



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/srv/conda/envs/saturn/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [77]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)

In [78]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 8

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

In [79]:
dataloader_train

In [83]:
val_loss, predictions, true_vals = evaluate(dataloader_train)

In [92]:
value1 = predictions.tolist()

In [93]:
train1['predict'] = value1

In [97]:
train1['predict_bins'] = train1['predict'].apply(lambda x : (pd.cut(x,bins=7,labels=[1,2,3,4,5,6,7])).tolist())

In [95]:
s = train1['labels'].unique()
s.sort()
class1 = list(s)

In [98]:
def dict1(x):
  dict1 = {}
  for i in range(0,20):
      dict1[class1[i]] = x[i]
  return dict1

In [100]:
train1['predict_dict'] = train1['predict'].apply(lambda x : dict1(x))

In [101]:
train1['predict_bin_dict'] = train1['predict_bins'].apply(lambda x : dict1(x))

In [102]:
train3 = train1[['Requirements','Class','predict_dict']]
train4 = train1[['Requirements','Class','predict_bin_dict']]

In [103]:
train3 = train3.drop('predict_dict', axis=1).join(pd.DataFrame(train3.predict_dict.values.tolist()))
train4 = train4.drop('predict_bin_dict', axis=1).join(pd.DataFrame(train4.predict_bin_dict.values.tolist()))

In [110]:
train3 = train3.drop_duplicates(subset=['Requirements','Class'])
train4 = train4.drop_duplicates(subset=['Requirements','Class'])

In [115]:
train3.to_csv('Prediction_scores_BERT.csv')
train4.to_csv('Prediction_ranks_BERT.csv')